In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns',None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

In [3]:
data = pd.read_json('Datasets/Property/properties.json',lines=True)

In [4]:
data.head()

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription
0,{'$oid': '5d2b113a43cbfd7c77a998f4'},room-1686123,14 m2,14,Rotterdam,https://resources.kamernet.nl/image/913b4b03-5...,done,{'$date': '2019-07-26T22:18:23.018+0000'},"[{'$date': '2019-07-14T11:25:46.511+0000'}, {'...",{'$date': '2019-07-14T11:25:46.511+0000'},Unfurnished,{'$date': '2019-07-26T22:18:23.142+0000'},51.896601,4.514993,3074HN,4w,Room,26-06-'19 - Indefinite period,500,,"€ 500,-",kamernet,West-Varkenoordseweg,https://kamernet.nl/en/for-rent/room-rotterdam...,50.0,\n € 50\n ...,500.0,\n € 500\n ...,"Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...","Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...",{'$date': '2019-07-22T07:10:41.849+0000'},Unknown,Mixed,Yes,true,Shared,None,16 years -\n 99 years,16 years -\n 99 years,1 person,Not important,Not important,Not important,Not important,Not important,"Room for rent in Rotterdam, West-Varkenoordse...",Room for rent in Rotterdam €500 | Kamernet,No,0,\n € 0\n ...,5,Shared,No,Shared,Huize west,4680711.0,21-07-2019,26-06-2019,https://resources.kamernet.nl/Content/images/s...,NaN
1,{'$oid': '5d2b113a43cbfd7c77a9991a'},studio-1691193,30 m2,30,Amsterdam,https://resources.kamernet.nl/image/5e11d6b5-8...,done,{'$date': '2019-08-10T22:28:46.099+0000'},"[{'$date': '2019-07-14T11:25:46.677+0000'}, {'...",{'$date': '2019-07-14T11:25:46.677+0000'},Furnished,{'$date': '2019-08-10T22:28:46.229+0000'},52.370200,4.920721,1018AS,4w,Studio,15-08-'19 - Indefinite period,950,Utilities incl.,"€ 950,- Utilities incl.",kamernet,Parelstraat,https://kamernet.nl/en/for-rent/studio-amsterd...,0.0,\n € 0\n ...,895.0,\n € 895\n ...,"Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...","Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...",{'$date': '2019-07-22T06:29:33.112+0000'},Unknown,Unknown,Yes,true,Own,Own,18 years -\n 99 years,18 years -\n 99 years,1 person,Not important,Not important,Not important,"Working student, Working","Working student, Working","Studio for rent in Amsterdam, Parelstraat, fo...",Studio for rent in Amsterdam €950 | Kamernet,No,0,\n € 0\n ...,None,Own,No,Own,Cor,1865530.0,20-07-2019,05-01-2012,https://resources.kamernet.nl/Content/images/p...,NaN
2,{'$oid': '5d2b113a43cbfd7c77a99931'},room-1690545,11 m2,11,Amsterdam,https://resources.kamernet.nl/image/74b93a27-a...,done,{'$date': '2019-10-02T22:00:33.141+0000'},"[{'$date': '2019-07-14T11:25:46.834+0000'}, {'...",{'$date': '2019-07-14T11:25:46.834+0000'},Furnished,{'$date': '2019-10-02T22:00:33.264+0000'},52.350880,4.854786,1075SB,09 Jul,Room,01-08-'19 - Indefinite period,1000,Utilities incl.,"€ 1000,- Utilities incl.",kamernet,Zeilstraat,https://kamernet.nl/en/for-rent/room-amsterdam...,NaN,\n -\n ...,1000.0,\n € 1000\n ...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,{'$date': '2019-07-21T08:44:32.816+0000'},Unknown,Mixed,Yes,true,Shared,Shared,16 years -\n 93 years,16 years -\n 93 years,1 person,Not important,Not i

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46722 entries, 0 to 46721
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   _id                          46722 non-null  object 
 1   externalId                   46722 non-null  object 
 2   areaRaw                      46722 non-null  object 
 3   areaSqm                      46722 non-null  int64  
 4   city                         46722 non-null  object 
 5   coverImageUrl                46722 non-null  object 
 6   crawlStatus                  46722 non-null  object 
 7   crawledAt                    46722 non-null  object 
 8   datesPublished               46722 non-null  object 
 9   firstSeenAt                  46722 non-null  object 
 10  furnish                      46722 non-null  object 
 11  lastSeenAt                   46722 non-null  object 
 12  latitude                     46722 non-null  float64
 13  longitude       

# Preprocessing

In [6]:
data['crawlStatus'].value_counts()

done           46622
unavailable      100
Name: crawlStatus, dtype: int64

In [7]:
# 100 come from missing values
data.query("crawlStatus == 'unavailable'")

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription
77,{'$oid': '5d2b116743cbfd7c77a9a94b'},room-1691706,13 m2,13,Den Haag,https://resources.kamernet.nl/image/5b50a59e-c...,unavailable,{'$date': '2019-07-14T11:26:31.513+0000'},[{'$date': '2019-07-14T11:26:31.636+0000'}],{'$date': '2019-07-14T11:26:31.636+0000'},Furnished,{'$date': '2019-07-14T11:26:31.636+0000'},52.060570,4.300775,2525ZA,21h,Room,01-08-'19 - 01-09-'19,500,Utilities incl.,"€ 500,- Utilities incl.",kamernet,Jacob Schorerlaan,https://kamernet.nl/en/for-rent/room-den-haag/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$date': '2019-07-22T07:13:33.837+0000'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,{'$oid': '5d2b116843cbfd7c77a9a9c8'},room-1691699,14 m2,14,Den Haag,https://resources.kamernet.nl/image/2fff4552-5...,unavailable,{'$date': '2019-07-16T06:02:01.690+0000'},"[{'$date': '2019-07-14T11:26:32.074+0000'}, {'...",{'$date': '2019-07-14T11:26:32.074+0000'},Furnished,{'$date': '2019-07-16T06:02:01.833+0000'},52.060570,4.300775,2525ZA,3d,Room,22-07-'19 - 27-12-'19,500,Utilities incl.,"€ 500,- Utilities incl.",kamernet,Jacob Schorerlaan,https://kamernet.nl/en/for-rent/room-den-haag/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$date': '2019-07-21T08:53:47.875+0000'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,{'$oid': '5d2b117043cbfd7c77a9ac67'},room-1691664,20 m2,20,Ubbena,https://resources.kamernet.nl/image/6160f67d-5...,unavailable,{'$date': '2019-07-16T06:02:12.010+0000'},"[{'$date': '2019-07-14T11:26:40.809+0000'}, {'...",{'$date': '2019-07-14T11:26:40.809+0000'},Furnished,{'$date': '2019-07-16T06:02:12.132+0000'},53.054733,6.589102,9492TG,3d,Room,13-07-'19 - Indefinite period,20,Utilities incl.,"€ 20,- Utilities incl.",kamernet,Taarloseweg,https://kamernet.nl/en/for-rent/room-ubbena/ta...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$date': '2019-07-22T07:10:25.755+0000'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,{'$oid': '5d2b117c43cbfd7c77a9b0dd'},room-1685795,18 m2,18,Groningen,https://resources.kamernet.nl/image/3bd7fa60-b...,unavailable,{'$date': '2019-07-15T00:01:48.442+0000'},"[{'$date': '2019-07-14T11:26:52.485+0000'}, {'...",{'$date': '2019-07-14T11:26:52.485+0000'},Uncarpeted,{'$date': '2019-07-15T00:01:48.564+0000'},53.232240,6.572627,9715AN,3w,Room,01-08-'19 - Indefinite period,355,Utilities incl.,"€ 355,- Utilities incl.",kamernet,Korreweg,https://kamernet.nl/en/for-rent/room-groningen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$date': '2019-07-21T09:08:27.206+0000'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,{'$oid': '5d2b11bf43cbfd7c77a9cbc1'},room-1691430,10 m2,10,Haarlem,https://resources.kamernet.nl/Content/images/p...,unavailable,{'$date': '2019-07-17T22:11:12.102+0000'},"[{'$date': '2019-07-14T11:27:59.797+0000'}, {'...",{'$date': '2019-07-14T11:27:59.797+0000'},Furnished,{'$date': '2019-07-17T22:11:12.223+0000'},52.365043,4.659657,2035VE,5d,Room,12-07-'19 - Indefinite period,350,Utilities incl.,"€ 350,- Utilities incl.",kamernet,Ekamastraat,https://kamernet.nl/en/f

In [23]:
data.query("crawlStatus == 'unavailable'").index

Int64Index([   77,    80,    91,   116,   231,   348,   484,   492,   537,
              553,   555,   570,   626,   740,   749,   840,   907,   930,
              932,   999,  1089,  1122,  1192,  1271,  1337,  1350,  1352,
             1565,  1615,  1942,  2038,  2346,  2598,  2704,  2995,  3041,
             3372,  3373,  3409,  3423,  3427,  3446,  3461,  3462,  3463,
             3488,  4176,  4202,  4213,  4342,  4347,  4398,  4443,  4482,
             4514,  4552,  4583,  4648,  4869,  4925,  4932,  4972,  5368,
             5378,  5641,  6019,  6020,  6275,  6361,  6493,  6643,  6768,
             6834,  6840,  6893,  6932,  6963,  6967,  8895, 10069, 11904,
            13503, 15265, 15739, 22198, 24546, 26579, 27214, 27377, 27378,
            27823, 34340, 35411, 38851, 42950, 43453, 44914, 45434, 45492,
            45520],
           dtype='int64')

In [8]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop missing rows
    bad_rows = df.query("crawlStatus == 'unavailable'").index
    df = df.drop(bad_rows, axis=0).reset_index(drop=True)
    
    # use only specific columns:
    df = df[[
        'areaSqm',
        'city',
        'furnish',
        'latitude',
        'longitude',
        'propertyType',
        'rent',
        'internet',
        'kitchen',
        'living',
        'pets',
        'shower',
        'smokingInside',
        'toilet'
    ]]
    
    # encode improper values
    df = df.replace({'': np.NaN, 'Unknown': np.NaN})
    
    # Fill missing values with mode of its column that are categorical
    missing_value_columns = df.columns[df.isna().sum() > 0]
    for column in missing_value_columns:
        df[column] = df[column].fillna(df[column].mode()[0])
    
    # split data into X and y
    y = df['rent']
    X = df.drop('rent', axis=1)
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, shuffle=True, random_state=1)
    
    # After train test split, replace return df with return X_train, X_test, y_train, y_test
    return X_train, X_test, y_train, y_test, X

In [9]:
X_train, X_test, y_train, y_test, X = preprocess_inputs(data)

In [7]:
X_train

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
29791,61,Rotterdam,Unfurnished,51.925125,4.486212,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
44827,45,Rotterdam,Unfurnished,51.893369,4.517075,Apartment,Yes,Own,Own,No,Own,No,Own
37089,105,Amsterdam,Furnished,52.376979,4.839116,Apartment,Yes,Own,Own,No,Own,No,Own
13269,20,Delft,Uncarpeted,51.996010,4.352954,Room,Yes,Shared,Shared,By mutual agreement,Shared,Not important,Shared
14654,44,Rotterdam,Furnished,51.891709,4.480317,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,11,Enschede,Unfurnished,52.234413,6.849043,Room,Yes,Shared,Shared,No,Shared,No,Shared
32511,21,Utrecht,Furnished,52.086425,5.125311,Room,Yes,Shared,Shared,No,Shared,No,Shared
5192,16,Groningen,Furnished,53.229623,6.524759,Room,Yes,Shared,Shared,No,Shared,No,Shared
12172,7,Utrecht,Furnished,52.102119,5.096010,Room,Yes,Shared,None,No,Shared,No,Shared


In [8]:
X_test

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
31410,21,Groningen,Unfurnished,53.232598,6.534303,Room,Yes,Shared,Shared,No,Shared,No,Shared
34454,50,Eindhoven,Furnished,51.426410,5.493228,Apartment,Yes,Own,Own,No,Own,No,Own
42517,10,Leeuwarden,Unfurnished,53.184591,5.785309,Room,Yes,Shared,Shared,No,Shared,No,Shared
36306,38,Nijmegen,Furnished,51.808167,5.830707,Room,No,Shared,Own,No,Shared,No,Shared
43045,16,Amsterdam,Furnished,52.373412,4.836547,Room,Yes,Shared,None,No,Own,Not important,Own
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42138,110,Haarlem,Uncarpeted,52.383507,4.641078,Apartment,No,Own,Own,No,Own,No,Own
36266,14,Amsterdam,Unfurnished,52.383644,4.913717,Room,No,Shared,Shared,No,Shared,No,Shared
7926,51,Haarlem,Unfurnished,52.373505,4.636374,Apartment,No,Own,Own,No,Own,No,Own
25058,22,Enschede,Unfurnished,52.220338,6.892566,Room,Yes,Shared,None,No,Shared,Not important,Shared


In [9]:
y_train

29791    1195
44827     950
37089    1500
13269     330
14654     845
         ... 
43723     310
32511     435
5192      325
12172     175
33003    1495
Name: rent, Length: 32635, dtype: int64

In [10]:
y_test

31410     400
34454     775
42517     345
36306     500
43045     800
         ... 
42138    1600
36266     795
7926     1250
25058     365
15087     475
Name: rent, Length: 13987, dtype: int64

# Preprocessing Checks

In [54]:
#X = preprocess_inputs(data)

In [14]:
X

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
0,14,Rotterdam,Unfurnished,51.896601,4.514993,Room,Yes,Shared,None,No,Shared,No,Shared
1,30,Amsterdam,Furnished,52.370200,4.920721,Studio,Yes,Own,Own,No,Own,No,Own
2,11,Amsterdam,Furnished,52.350880,4.854786,Room,Yes,Shared,Shared,Yes,Shared,Yes,Shared
3,16,Assen,Unfurnished,53.013494,6.561012,Room,Yes,Shared,None,No,Shared,Yes,Shared
4,22,Rotterdam,Unfurnished,51.932871,4.479732,Room,Yes,Own,Own,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46617,28,Rotterdam,Furnished,51.928624,4.507187,Room,Yes,Shared,Shared,No,Shared,No,Shared
46618,16,Harmelen,Furnished,52.086568,4.959942,Room,Yes,Shared,Shared,No,Shared,No,Shared
46619,30,Rotterdam,Furnished,51.928624,4.507187,Room,Yes,Shared,Shared,No,Shared,No,Shared
46620,35,Rotterdam,Furnished,51.928624,4.507187,Room,Yes,Shared,Shared,No,Shared,No,Shared


In [56]:
# continious Column check
X.describe()

,areaSqm,latitude,longitude,rent
count,46622.000000,46622.000000,46622.000000,46622.000000
mean,31.636974,52.201675,5.315371,667.974111
std,29.877284,0.517286,0.798836,416.830406
min,6.000000,50.770041,3.410016,1.000000
25%,14.000000,51.925374,4.712789,395.000000
50%,20.000000,52.162522,5.083407,550.000000
75%,40.000000,52.370420,5.896593,800.000000
max,675.000000,53.434608,7.206637,5999.000000


In [57]:
X.isna().sum()

areaSqm          0
city             0
furnish          0
latitude         0
longitude        0
propertyType     0
rent             0
internet         0
kitchen          0
living           0
pets             0
shower           0
smokingInside    0
toilet           0
dtype: int64

In [10]:
# Checking Categorical Column
# Making dictionary
{column: list(X[column].unique()) for column in X.select_dtypes('object').columns}

# will replace the unknown and null values with NaN

{'city': ['Rotterdam',
  'Amsterdam',
  'Assen',
  'Groningen',
  'Zeist',
  'Maastricht',
  'Callantsoog',
  'Alphen aan den Rijn',
  'Tilburg',
  'Enschede',
  'Leeuwarden',
  'Eindhoven',
  'Wageningen',
  'Diemen',
  'Utrecht',
  'Almere',
  'Alkmaar',
  'Harderwijk',
  'Hilversum',
  'Delft',
  'Den Bosch',
  'Stoutenburg',
  'Leiden',
  'Den Haag',
  'Boxtel',
  'Badhoevedorp',
  'Veenendaal',
  'Amstelveen',
  'Nijmegen',
  'Venlo',
  'Zwolle',
  'Ubbena',
  'Arnhem',
  'Leimuiden',
  'Riel',
  'Nieuwegein',
  'Haren Gn',
  'Uitgeest',
  'Beverwijk',
  'Ede',
  'Nijkerk',
  'Amersfoort',
  'Loosdrecht',
  'Apeldoorn',
  'Vaals',
  'Velp',
  'Vlaardingen',
  'Montfoort',
  'Heemstede',
  'Breda',
  'Purmerend',
  'Baarn',
  'Spijkenisse',
  'Deventer',
  'Hengelo',
  'Capelle aan den IJssel',
  'Bovenkarspel',
  'Weesp',
  'Harskamp',
  'Zeeland',
  'Waalre',
  'IJsselstein',
  'Pijnacker',
  'Sittard',
  'Putten',
  'Vlissingen',
  'Haarlem',
  'Rijswijk',
  'Zandvoort',
  'Zutp

In [11]:
# Checking Categorical Column
# Making dictionary
{column: list(X[column].unique()) for column in X.select_dtypes('object').columns}

{'city': ['Rotterdam',
  'Amsterdam',
  'Assen',
  'Groningen',
  'Zeist',
  'Maastricht',
  'Callantsoog',
  'Alphen aan den Rijn',
  'Tilburg',
  'Enschede',
  'Leeuwarden',
  'Eindhoven',
  'Wageningen',
  'Diemen',
  'Utrecht',
  'Almere',
  'Alkmaar',
  'Harderwijk',
  'Hilversum',
  'Delft',
  'Den Bosch',
  'Stoutenburg',
  'Leiden',
  'Den Haag',
  'Boxtel',
  'Badhoevedorp',
  'Veenendaal',
  'Amstelveen',
  'Nijmegen',
  'Venlo',
  'Zwolle',
  'Ubbena',
  'Arnhem',
  'Leimuiden',
  'Riel',
  'Nieuwegein',
  'Haren Gn',
  'Uitgeest',
  'Beverwijk',
  'Ede',
  'Nijkerk',
  'Amersfoort',
  'Loosdrecht',
  'Apeldoorn',
  'Vaals',
  'Velp',
  'Vlaardingen',
  'Montfoort',
  'Heemstede',
  'Breda',
  'Purmerend',
  'Baarn',
  'Spijkenisse',
  'Deventer',
  'Hengelo',
  'Capelle aan den IJssel',
  'Bovenkarspel',
  'Weesp',
  'Harskamp',
  'Zeeland',
  'Waalre',
  'IJsselstein',
  'Pijnacker',
  'Sittard',
  'Putten',
  'Vlissingen',
  'Haarlem',
  'Rijswijk',
  'Zandvoort',
  'Zutp

In [58]:
X.isna().mean()

# if mean is over 25%, safe to drop column
# for this project, any thing greater than 0 will be filled with mode of each column

areaSqm          0.0
city             0.0
furnish          0.0
latitude         0.0
longitude        0.0
propertyType     0.0
rent             0.0
internet         0.0
kitchen          0.0
living           0.0
pets             0.0
shower           0.0
smokingInside    0.0
toilet           0.0
dtype: float64

In [46]:
X['living'].mode()

0    Shared
dtype: object

In [47]:
X['living'].mode()[0]

'Shared'

In [50]:
X.columns

Index(['areaSqm', 'city', 'furnish', 'latitude', 'longitude', 'propertyType',
       'rent', 'internet', 'kitchen', 'living', 'pets', 'shower',
       'smokingInside', 'toilet'],
      dtype='object')

In [51]:
X.isna().sum() > 0

areaSqm          False
city             False
furnish           True
latitude         False
longitude        False
propertyType     False
rent             False
internet          True
kitchen           True
living            True
pets             False
shower            True
smokingInside    False
toilet            True
dtype: bool

In [52]:
X.columns[X.isna().sum() > 0] # this only filters coumns that have missing values

Index(['furnish', 'internet', 'kitchen', 'living', 'shower', 'toilet'], dtype='object')

# Building Pipeline and Training

In [12]:
{column: len(X[column].unique()) for column in X.columns}

{'areaSqm': 227,
 'city': 737,
 'furnish': 3,
 'latitude': 18154,
 'longitude': 18262,
 'propertyType': 5,
 'internet': 2,
 'kitchen': 3,
 'living': 3,
 'pets': 3,
 'shower': 3,
 'smokingInside': 3,
 'toilet': 3}

In [13]:
{column: len(X[column].unique()) for column in X.select_dtypes('object').columns}

# look for any values are are 2. these will be used with single columns with Ordinal encoding otherwise,
# they will cause duplication. he rest can use  OneHot encoding.

{'city': 737,
 'furnish': 3,
 'propertyType': 5,
 'internet': 2,
 'kitchen': 3,
 'living': 3,
 'pets': 3,
 'shower': 3,
 'smokingInside': 3,
 'toilet': 3}

In [14]:
pd.get_dummies(X['propertyType']) 

,Anti-squat,Apartment,Room,Student residence,Studio
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
46617,0,0,1,0,0
46618,0,0,1,0,0
46619,0,0,1,0,0
46620,0,0,1,0,0


In [15]:
# can use this way but we can use ordinal encoder
pd.get_dummies(X['internet']).drop('No', axis=1)

,Yes
0,1
1,1
2,1
3,1
4,1
...,...
46617,1
46618,1
46619,1
46620,1


In [16]:
# Need to convert categorical columns to numerical ones for data processing

In [17]:
nominal_features = [
        'city',
        'furnish',
        'propertyType',
        'kitchen',
        'living',
        'pets',
        'shower',
        'smokingInside',
        'toilet'
    ]

binary_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())
])

nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, ['internet']),
    ('nominal', nominal_transformer, nominal_features)
], remainder = 'passthrough')

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [18]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('binary',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['internet']),
                                                 ('nominal',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['city', 'furnish',
                                                   'propertyType', 'kitchen',
                                                   'living', 'pets', 'shower',
                                              

# Results

In [19]:
y_pred = model.predict(X_test) #issue with unknown categories. can be fixed by setting  handle_unknown='ignore' in OneHotEncoder

In [20]:
y_pred

array([ 430.1       ,  948.24      ,  341.31      , ..., 1327.54      ,
        352.91666667,  438.12      ])

In [21]:
y_test

31410     400
34454     775
42517     345
36306     500
43045     800
         ... 
42138    1600
36266     795
7926     1250
25058     365
15087     475
Name: rent, Length: 13987, dtype: int64

In [22]:
# average error accross all preds
y_test - y_pred

31410    -30.100000
34454   -173.240000
42517      3.690000
36306    -10.970000
43045    -61.176667
            ...    
42138     17.610000
36266     22.237500
7926     -77.540000
25058     12.083333
15087     36.880000
Name: rent, Length: 13987, dtype: float64

In [23]:
np.mean(y_test - y_pred) # average error: some waht close to 0. Not a good way to tell how model is doing

-3.930366918672851

In [24]:
(y_test - y_pred)**2 # squared to make them positive

31410      906.010000
34454    30012.097600
42517       13.616100
36306      120.340900
43045     3742.584544
             ...     
42138      310.112100
36266      494.506406
7926      6012.451600
25058      146.006944
15087     1360.134400
Name: rent, Length: 13987, dtype: float64

In [25]:
np.mean((y_test - y_pred)**2) # mean squared error accross all examples. This is squared Euros

24845.579189714234

In [26]:
np.sqrt(np.mean((y_test - y_pred)**2)) # rmse: get original average error in Euros. We are 156 euro off from rent prediction.

157.62480512189137

In [27]:
y_test.describe() # distribution of variable:

count    13987.000000
mean       664.911561
std        413.872062
min          1.000000
25%        390.000000
50%        550.000000
75%        800.000000
max       5000.000000
Name: rent, dtype: float64

In [28]:
# Analyze with R^2 score. Its a way of comparing the model to the baseline model. (What model would you use if you had no input data avaiable to you).
y_test.mean()

664.9115607349681

In [29]:
np.sum((y_test - y_test.mean())**2) # These are errors while calculating errors for the baseline model

2395663115.600629

In [30]:
np.sum((y_test - y_pred)**2) # These are errors for our model

347515116.12653244

In [31]:
# Comparison
(np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
# Results give us something close to 0 meaning we have very low error compared to the baseline.
# If results have higher error compared to baseline then figure would be higher close to infinite.
# If its the same then its equal to 1. Meaning our model is similar to baseline model.

0.14506009374335804

In [32]:
# R^2 Score: range: [-inf, 1]
1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
# Result: we have have 86% reduction in error from the baselines error

0.854939906256642

# Scores

In [33]:
# RMSE: This score is an absolute metric. Its evaluates your model absolutely in terms of units of the orginal target variable.
rmse = np.sqrt(np.mean((y_test - y_pred)**2))

In [34]:
# R^2: comparative metric, this score compares your model to another baseline model
r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))

In [35]:
print("     RMSE: {:.2f}".format(rmse))
print("R^2 Score: {:.5f}".format(r2))

     RMSE: 157.62
R^2 Score: 0.85494


In [36]:
# We can see that we are 156.91 euros off.
# And we are 86% better than the baseline model.

# Predictions

In [37]:
df2 = df2([
    
])

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
31410,21,Groningen,Unfurnished,53.232598,6.534303,Room,Yes,Shared,Shared,No,Shared,No,Shared
34454,50,Eindhoven,Furnished,51.426410,5.493228,Apartment,Yes,Own,Own,No,Own,No,Own
42517,10,Leeuwarden,Unfurnished,53.184591,5.785309,Room,Yes,Shared,Shared,No,Shared,No,Shared
36306,38,Nijmegen,Furnished,51.808167,5.830707,Room,No,Shared,Own,No,Shared,No,Shared
43045,16,Amsterdam,Furnished,52.373412,4.836547,Room,Yes,Shared,None,No,Own,Not important,Own
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42138,110,Haarlem,Uncarpeted,52.383507,4.641078,Apartment,No,Own,Own,No,Own,No,Own
36266,14,Amsterdam,Unfurnished,52.383644,4.913717,Room,No,Shared,Shared,No,Shared,No,Shared
7926,51,Haarlem,Unfurnished,52.373505,4.636374,Apartment,No,Own,Own,No,Own,No,Own
25058,22,Enschede,Unfurnished,52.220338,6.892566,Room,Yes,Shared,None,No,Shared,Not important,Shared
